In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# # Initialize the video capture object
# video_path = '/home/ikharitonov/Desktop/20204321_343_5.avi'
# cap = cv2.VideoCapture(video_path)

# # Initialize the background subtractor
# fgbg = cv2.createBackgroundSubtractorMOG2(history=5000, varThreshold=50, detectShadows=True)

# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Apply the background subtractor to get the foreground mask
#     fgmask = fgbg.apply(frame)

#     # Remove noise by applying a series of erosions and dilations
#     kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
#     fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
#     fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel)

#     # Apply GaussianBlur to reduce noise and improve circle detection
#     blurred = cv2.GaussianBlur(fgmask, (9, 9), 2, 2)

#     # Detect circles in the blurred image
#     circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1, minDist=50, param1=50, param2=30, minRadius=5, maxRadius=50)

#     if circles is not None:
#         circles = np.uint8(np.around(circles))
#         for i in circles[0, :]:
#             # Draw the outer circle
#             cv2.circle(frame, (i[0], i[1]), i[2], (0, 255, 0), 2)
#             # Draw the center of the circle
#             cv2.circle(frame, (i[0], i[1]), 2, (0, 0, 255), 3)

#     # Display the result
#     cv2.imshow('Frame', frame)
#     cv2.imshow('Foreground Mask', fgmask)

#     # Break the loop if 'q' is pressed
#     if cv2.waitKey(30) & 0xFF == ord('q'):
#         break

# # Release the video capture object and close all OpenCV windows
# cap.release()
# cv2.destroyAllWindows()


In [ ]:
# Loading the video and selecting one frame
video_path = '/home/ikharitonov/Desktop/20204321_343_5.avi'
cap = cv2.VideoCapture(video_path)
frame_number=1000
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number-1)
res, frame = cap.read()
frame.shape

In [ ]:
# Grayscaling and plotting the histogram of the image
img = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

plt.hist(img.flatten(), bins=20)
plt.show()

In [ ]:
# Thresholding the image with max val minus one standard deviation
threshold = img.max() - img.std()
ret, th1 = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)

plt.imshow(th1)
plt.show()

In [ ]:
# Trying to fit circles to the image
circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, dp=1, minDist=50, param1=50, param2=30, minRadius=10, maxRadius=50)
circles

In [ ]:
plt.imshow(frame[:,:,:])
for c in circles[0,:]:
    circle_to_plot = plt.Circle((c[0], c[1]), c[2], color='r', alpha=0.4)
    plt.gca().add_patch(circle_to_plot)
plt.show()

In [ ]:
# Finding contours in the thresholded image
contours, _ = cv2.findContours(th1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
len(contours)

In [ ]:
# Plotting detected contours
for contour in contours:
    # Approximate the contour to a circle
    (x, y), r = cv2.minEnclosingCircle(contour)
    center = (int(x), int(y))
    radius = int(r)
    
    # Calculate circularity: (4 * π * Area) / Perimeter^2
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    if perimeter > 0:
        circularity = (4 * np.pi * area) / (perimeter * perimeter)
        # Check if the circularity is close to 1 (perfect circle)
        if 0.7 < circularity < 1.3:
            circular_contour = contour
            break

# Draw the detected circle on the image (if found)
if circular_contour is not None:
    output_image = cv2.cvtColor(th1, cv2.COLOR_GRAY2BGR)
    cv2.circle(output_image, center, radius, (0, 255, 0), 2)
else:
    output_image = thresholded_image.copy()

# Display the result
plt.figure(figsize=(6, 6))
plt.imshow(output_image, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
# Collecting contour data (circularity, center point coordinates, radius, area) into an array
contour_data = []

for i, contour in enumerate(contours):
    (x, y), r = cv2.minEnclosingCircle(contour)
    center = (int(x), int(y))
    radius = int(r)
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    if perimeter > 0:
        circularity = (4 * np.pi * area) / (perimeter * perimeter)
        contour_data.append((i,circularity, center[0], center[1], radius, area))
contour_data = np.array(contour_data)

In [ ]:
plt.title('Circularity vs. area of contours')
plt.scatter(contour_data[:,1], contour_data[:,5])
plt.xlabel('circularity')
plt.ylabel('area')
plt.show()

plt.title('Circularity histogram')
plt.hist(contour_data[:,1])
plt.xlabel('circularity')
plt.show()

plt.title('Area histogram')
plt.hist(contour_data[:,5])
plt.xlabel('area')
plt.show()

In [ ]:
np.mean(contour_data[:,5])

In [ ]:
# Trying to find the most circular circle
max_circularity_ind = np.argmax(contour_data[:,1])
center = (int(contour_data[max_circularity_ind][2]), int(contour_data[max_circularity_ind][3]))
radius = int(contour_data[max_circularity_ind][4])
center

In [ ]:
# Searching for the contour by eye in the area-circularity plot
min_area = 50
max_area = 100

contour_data_ind = np.where((contour_data[:,5] >= min_area) & (contour_data[:,5] <= max_area))[0][0]
contour_ind = int(contour_data[contour_data_ind,0])

cv2.ellipse(frame, cv2.fitEllipse(contours[contour_ind]), (0, 255, 0), 1)

plt.figure(figsize=(8, 8))
plt.imshow(frame)
plt.axis('off')
plt.show()

In [ ]:
# Filtering the contours by the number of associated points
numbers_of_points = np.array([(i, c.shape[0]) for i,c in enumerate(contours)]) # [[contour_ind, number_of_points], ...]
countour_inds_above_5 = np.where(numbers_of_points[:,1] >= 5)
contours_above_5 = list(np.array(contours)[countour_inds_above_5])

In [ ]:
# Fitting ellipses to contours with more than 5 points
fitted_ellipses = [cv2.fitEllipse(c) for c in contours_above_5]
fitted_ellipses

In [ ]:
for e in fitted_ellipses:
    cv2.ellipse(frame, e, (0, 255, 0), 1)

# Display the result
plt.figure(figsize=(8, 8))
# plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.imshow(frame)
plt.axis('off')
plt.show()

In [ ]:
def euclidean_distance(point1, point2):
    return np.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)

def get_ellipse_xy(contour):
    (x,y),_,_ = cv2.fitEllipse(contour)
    return (x,y)

def heuristic_contour_selection(selected_contours):
    # Check that each element in the input list of contours has at least five points
    # Is located at the coordinate with the lowest distance from the center of the image
    selected_contours = [c for c in selected_contours if c.shape[0] >= 5]
    center_point = (img.shape[0]//2, img.shape[1]//2)
    distances = [euclidean_distance(center_point, get_ellipse_xy(c)) for c in selected_contours]

    return selected_contours[np.argmin(distances)]

def get_coordinates_method_two(contours):
    
    contour_data = []

    for i, contour in enumerate(contours):
        (x, y), r = cv2.minEnclosingCircle(contour)
        center = (int(x), int(y))
        radius = int(r)
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        if perimeter > 0:
            circularity = (4 * np.pi * area) / (perimeter * perimeter)
            contour_data.append((i,circularity, center[0], center[1], radius, area))
    contour_data = np.array(contour_data)
    
    # Selecting the contour with maximum area out of all contours filtered by circularity value
    threshold_circularity = 0.6
    selected_max_circularity_contours = contour_data[np.where(contour_data[:,1] > threshold_circularity), :][0]
    # max_area_contour_ind = int(selected_max_circularity_contours[np.argmax(selected_max_circularity_contours[:,5]), 0])
    max_contour_inds_desc_order = selected_max_circularity_contours[np.argsort(selected_max_circularity_contours[:,5])[::-1], 0].astype(int)
    selected_contours = list(np.array(contours, dtype=object)[max_contour_inds_desc_order])
    
    return cv2.fitEllipse(heuristic_contour_selection(selected_contours))
    
#     (x,y),_,_ = cv2.fitEllipse(contours[max_area_contour_ind])
    
#     return x,y
#     if contours[max_area_contour_ind].shape[0] < 5:
#         return None
#     else:
#         return cv2.fitEllipse(contours[max_area_contour_ind])

In [ ]:
# Plotting
video_path = '/home/ikharitonov/Desktop/20204321_343_5.avi'
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    threshold = img.max() - img.std()
    ret, th1 = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
    if not ret:
        break

    contours, _ = cv2.findContours(th1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    ellipse = get_coordinates_method_two(contours)
    if not ellipse:
        pass
    else:
        cv2.ellipse(frame, ellipse, (0, 255, 0), 2)

    # Display the result
    cv2.imshow('Frame', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
def euclidean_distance(point1, point2):
    return np.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)

def get_ellipse_xy(contour):
    (x,y),_,_ = cv2.fitEllipse(contour)
    return (x,y)

def heuristic_contour_selection(selected_contours):
    # Check that each element in the input list of contours has at least five points
    # Is located at the coordinate with the lowest distance from the center of the image
    selected_contours = [c for c in selected_contours if c.shape[0] >= 5]
    center_point = (img.shape[0]//2, img.shape[1]//2)
    distances = [euclidean_distance(center_point, get_ellipse_xy(c)) for c in selected_contours]

#     return selected_contours[np.argmin(distances)]
    return list(np.array(selected_contours)[np.argsort(distances)])

def get_coordinates_method_two(contours):
    
    contour_data = []

    for i, contour in enumerate(contours):
        (x, y), r = cv2.minEnclosingCircle(contour)
        center = (int(x), int(y))
        radius = int(r)
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        if perimeter > 0:
            circularity = (4 * np.pi * area) / (perimeter * perimeter)
            contour_data.append((i,circularity, center[0], center[1], radius, area))
    contour_data = np.array(contour_data)
    
    # Selecting the contour with maximum area out of all contours filtered by circularity value
    threshold_circularity = 0.6
    selected_max_circularity_contours = contour_data[np.where(contour_data[:,1] > threshold_circularity), :][0]
    # max_area_contour_ind = int(selected_max_circularity_contours[np.argmax(selected_max_circularity_contours[:,5]), 0])
    max_contour_inds_desc_order = selected_max_circularity_contours[np.argsort(selected_max_circularity_contours[:,5])[::-1], 0].astype(int)
    selected_contours = list(np.array(contours, dtype=object)[max_contour_inds_desc_order])
    
    return [cv2.fitEllipse(c) for c in heuristic_contour_selection(selected_contours)]
    
#     (x,y),_,_ = cv2.fitEllipse(contours[max_area_contour_ind])
    
#     return x,y
#     if contours[max_area_contour_ind].shape[0] < 5:
#         return None
#     else:
#         return cv2.fitEllipse(contours[max_area_contour_ind])

def run_pipeline(videopath):
    output_ellipse_data = []
    
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        img = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        threshold = img.max() - img.std()
        ret, th1 = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
        if not ret:
            break

        contours, _ = cv2.findContours(th1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        output_ellipse_data.append(get_coordinates_method_two(contours))
#         if not ellipse:
#             pass
#         else:
#             cv2.ellipse(frame, ellipse, (0, 255, 0), 2)

#         # Display the result
#         cv2.imshow('Frame', frame)


    # Release the video capture object and close all OpenCV windows
    cap.release()
#     cv2.destroyAllWindows()
    return output_ellipse_data


output_ellipse_data = run_pipeline('/home/ikharitonov/Desktop/20204321_343_5.avi')

In [ ]:
output_ellipse_data

In [ ]:
np.save('/home/ikharitonov/Desktop/20204321_343_5_ellipse_data.npy', np.array(output_ellipse_data))

In [ ]:
number_of_ellipses_per_frame = []
for ellipse_list in output_ellipse_data:
    number_of_ellipses_per_frame.append(len(ellipse_list))

plt.hist(number_of_ellipses_per_frame, bins=50)
plt.xlabel('number of ellipses detected per frame')
plt.show()

In [ ]:
closest_ellipse_x_timeseries = np.zeros(len(output_ellipse_data))
closest_ellipse_y_timeseries = np.zeros(len(output_ellipse_data))

for i in range(len(output_ellipse_data)):
    if len(output_ellipse_data[i]) > 0:
        closest_ellipse_x_timeseries[i] = output_ellipse_data[i][0][0][0]
        closest_ellipse_y_timeseries[i] = output_ellipse_data[i][0][0][1]

In [ ]:
closest_ellipse_x_timeseries

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(closest_ellipse_x_timeseries)
# plt.xlim([0,2000])
plt.show()
plt.figure(figsize=(10,5))
plt.plot(closest_ellipse_y_timeseries)
plt.show()